- At the 1st round of cross-validatiaon, Adventitia - coronary showed poor prediction accuracy (f1 score < 0.5) compared to other labels.
- Therefore, will remove from the label list and re-try cross validataion

# Import required modules

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import os

In [2]:
import tissuetypist as tt

In [3]:
import importlib
importlib.reload(tt)

<module 'tissuetypist' from '/nfs/team205/kk18/github/TissueTypist/tissuetypist/__init__.py'>

In [4]:
sc.settings.verbosity = 1          # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80)

# Variables

In [5]:
dir_to_results = '/nfs/team205/kk18/notebooks/P48_Foetal-heart_Visium/notebooks/TissueTypist/tissuetypist_package/cross_validation_results/7Apr2025_without-CoronaryAdv'
# make the directory if not exist
if not os.path.exists(dir_to_results):
    os.makedirs(dir_to_results)

# Read in pre-processed dataframe

In [6]:
data = pd.read_pickle('/nfs/team205/kk18/notebooks/P48_Foetal-heart_Visium/notebooks/TissueTypist/tissuetypist_package/preprocessed_df/visiumsd_oct.pkl')
print(data.shape)
data.head()

(19590, 5221)


,ITGA2_own,ZBTB42_own,MATN2_own,GLT8D2_own,CFH_own,DHRS3_own,ITM2C_own,SNCA_own,MYL2_own,CDK18_own,...,SHISA3_neighbour-max,GATA6-AS1_neighbour-max,LIMA1_neighbour-max,section,x,y,tissue,n_neighbours,is_edge,distance_to_edge
spot_id,,,,,,,,,,,,,,,,,,,,,
HCAHeartST10298114_AAACAAGTATCTCCCA-1,0.0,0.0,0.576221,0.000000,0.000000,0.576221,0.576221,0.0,3.627727,0.000000,...,0.0,1.061216,0.488556,HCAHeartST10298114,102,50,Endocardium - Ventricular,4.0,False,1.999961
HCAHeartST10298114_AAACATTTCCCGGATT-1,0.0,0.0,0.511360,0.000000,0.000000,0.000000,1.099502,0.0,3.456557,0.511360,...,0.0,1.236689,0.000000,HCAHeartST10298114,97,61,Ventricle - Compact,4.0,False,1.105358
HCAHeartST10298114_AAACGAGACGGTTGAT-1,0.0,0.0,0.000000,0.000000,0.000000,1.449640,0.735820,0.0,4.125242,0.434115,...,0.0,0.874152,0.000000,HCAHeartST10298114,79,35,Ventricle - Compact,4.0,False,1.105358
HCAHeartST10298114_AAACTCGTGATATAAG-1,0.0,0.0,1.497885,0.833815,0.624898,0.624898,1.753418,0.0,1.006538,0.000000,...,0.0,0.932834,1.116977,HCAHeartST10298114,113,23,Great vessel,4.0,False,0.887303
HCAHeartST10298114_AAACTGCTGGCTCCAA-1,0.0,0.0,0.665308,0.000000,0.000000,0.386992,1.061319,0.0,3.379385,0.000000,...,0.0,1.038130,0.000000,HCAHeartST10298114,67,45,Ventricle - Compact,3.0,True,0.000000


In [7]:
# remove coronary adventitia spots
data = data[data['tissue']!='Adventitia - Coronary']
print(data.shape)

(19179, 5221)


# Cross validation

In [8]:
# Training data: weight to the seighbour spot data
weight2neighbour_list = [0,0.1,0.3,1,3]
weight2isedge_list = [0,1,5]
edge_feature_list = ['distance_to_edge','is_edge']

In [9]:
# prepare features
own_features = [x for x in data.columns if '_own' in x]
neighbour_features = [x for x in data.columns if '_neighbour-max' in x]
print(len(own_features))
print(own_features[:5])
print(len(neighbour_features))
print(neighbour_features[:5])

2607
['ITGA2_own', 'ZBTB42_own', 'MATN2_own', 'GLT8D2_own', 'CFH_own']
2607
['ITGA2_neighbour-max', 'ZBTB42_neighbour-max', 'MATN2_neighbour-max', 'GLT8D2_neighbour-max', 'CFH_neighbour-max']


In [10]:
for edge_feature in edge_feature_list:
    all_fold_results = []
    f1_score_class_results = []
    for neighbour_weight in weight2neighbour_list:
        for edge_weight in weight2isedge_list:
            print(f"Weight to neighbour spots: {neighbour_weight}, Weight to {edge_feature}: {edge_weight}")
            df_folds, f1_score_class_df = tt.cross_validation(data,
                                                              own_features,
                                                              neighbour_features,
                                                              edge_feature,
                                                              neighbour_weight,
                                                              edge_weight,
                                                              tissue_col='tissue',
                                                             )
            all_fold_results.append(df_folds)
            f1_score_class_results.append(f1_score_class_df)

    # Concatenate all factors into one tidy DataFrame
    results_per_fold = pd.concat(all_fold_results, ignore_index=True)
    f1_score_class_per_fold = pd.concat(f1_score_class_results, ignore_index=True)
    # save
    results_per_fold.to_csv(f'{dir_to_results}/{edge_feature}_results_per_fold.csv')
    f1_score_class_per_fold.to_csv(f'{dir_to_results}/{edge_feature}_f1_score_class_per_fold.csv')

Weight to neighbour spots: 0, Weight to distance_to_edge: 0


Weight to neighbour spots: 0, Weight to distance_to_edge: 1


Weight to neighbour spots: 0, Weight to distance_to_edge: 5


Weight to neighbour spots: 0.1, Weight to distance_to_edge: 0


Weight to neighbour spots: 0.1, Weight to distance_to_edge: 1


...

Weight to neighbour spots: 1, Weight to is_edge: 1


Weight to neighbour spots: 1, Weight to is_edge: 5


Weight to neighbour spots: 3, Weight to is_edge: 0


Weight to neighbour spots: 3, Weight to is_edge: 1


Weight to neighbour spots: 3, Weight to is_edge: 5
